**IMPORTAMOS LIBRERIAS**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.tree import plot_tree
from sklearn import metrics
import numpy as np

**IMPORTAMOS AMBOS CSV**        
    - Train           
    - Test

In [2]:
train_c = pd.read_csv("../Data/train_c.csv")
test_c = pd.read_csv("../Data/test_c.csv")

In [3]:
train_c.head(1)

,id,carat,cut,color,clarity,depth,table,price
0,0,1.14,5,4,6,61.0,56.0,9013


In [4]:
test_c.head(1)

,id,carat,cut,color,clarity,depth,table
0,0,1.0,1,1,2,56.3,64.0


**SELECCIONAMOS PREDICTOR Y RESULTADOS**

In [5]:
X = train_c.drop(['price','id'], axis = 1)
y = train_c.price

In [6]:
X.head(1)

,carat,cut,color,clarity,depth,table
0,1.14,5,4,6,61.0,56.0


In [7]:
y[:1]

0    9013
Name: price, dtype: int64

**CREAMOS TRAIN_TEST_SPLIT CON UN 80% - 20%**

In [8]:
# creamos train_test_split

X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2)

In [9]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(32364, 6) (8091, 6) (32364,) (8091,)


**PROBAREMOS VARIOS MODELOS**

In [10]:
models = {
    'LinearRegression': LinReg(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'SGDRegressor': SGDRegressor(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),}

**LOS ENTRENAMOS**

In [11]:
for name, model in models.items():
    print("Entrenamos modelo: ", name)
    model.fit(X_train, y_train)

Entrenamos modelo:  LinearRegression
Entrenamos modelo:  Ridge
Entrenamos modelo:  Lasso
Entrenamos modelo:  SGDRegressor
Entrenamos modelo:  KNeighborsRegressor
Entrenamos modelo:  GradientBoostingRegressor
Entrenamos modelo:  DecisionTreeRegressor
Entrenamos modelo:  RandomForestRegressor


**VEMOS LOS RESULTADOS**

In [12]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f' RMSE =>',np.sqrt(metrics.mean_squared_error(y_test, y_pred)),f'    {model}')

 RMSE => 1232.8047441693873     LinearRegression()
 RMSE => 1232.7893743748825     Ridge()
 RMSE => 1232.7698763482415     Lasso()
 RMSE => 21573210.359631076     SGDRegressor()
 RMSE => 1891.06620723426     KNeighborsRegressor()
 RMSE => 628.5731052707508     GradientBoostingRegressor()
 RMSE => 739.9170008565437     DecisionTreeRegressor()
 RMSE => 562.4901932500125     RandomForestRegressor()


**NUESTRO MEJOR ERROR LO ENCONTRAMOS EN RANDOM FOREST***

**LO ENTRENAMOS Y HACEMOS NUESTRA PREDICCIÓN**

In [13]:
forest = RandomForestRegressor()
forest.fit(X_train,y_train)

RandomForestRegressor()

In [14]:
woid = test_c.drop(["id"], axis=1)

In [15]:
y_pred = forest.predict(woid)

**INCLUIMOS LA PREDICCIÓN EN NUESTRO DATAFRAME**

In [16]:
test_c["y_pred"] = y_pred

In [17]:
test_c.head(1)

,id,carat,cut,color,clarity,depth,table,y_pred
0,0,1.0,1,1,2,56.3,64.0,3329.87


**SELECCIONAMOS LAS COLUMNAS QUE KAGGLE NOS SOLICITA**

In [18]:
submission1 = test_c[["id","y_pred"]]
submission1.head(1)

,id,y_pred
0,0,3329.87


**SETEAMOS ID COMO INDEX**

In [19]:
test1 = submission1.set_index('id')
test1.head(1)

,y_pred
id,
0,3329.87


**CAMBIAMOS EL NOMBRE DE LA COLUMNA**

In [20]:
test1.columns = ['price']
test1.head(1)

,price
id,
0,3329.87


**EXPORTAMOS LA SUBMISSION EN FORMATO CSV**

In [21]:
test1.to_csv("../Data/submission1.csv")

          ---------------------------------------- X ---------------------------------------- 

**VAMOS A INTENTAR EL MODELO GRID_SEARCH_CV**               
**BUSCAMOS LOS MEJORES PARAMETROS**

In [25]:
params = {'max_depth': [20, 40, 60, 80, 100],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [100, 140, 180]}

In [28]:
gscv = GridSearchCV(forest,params, verbose=1)

In [29]:
gscv.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [20, 40, 60, 80, 100],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'n_estimators': [100, 140, 180]},
             verbose=1)

In [30]:
print(gscv.best_params_)

{'max_depth': 40, 'max_features': 'auto', 'min_samples_leaf': 4, 'n_estimators': 180}


**REPETIMOS EL MODELO INCLUYENDO LOS MEJORES PARÁMETROS**

In [31]:
forest = RandomForestRegressor(max_depth = 40, min_samples_leaf = 4, n_estimators = 180)

In [32]:
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=40, min_samples_leaf=4, n_estimators=180)

In [33]:
y_pred = forest.predict(woid)

In [35]:
print('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

ValueError: Found input variables with inconsistent numbers of samples: [8091, 13485]